In [1]:
import numpy as np
from scipy.linalg import solve

In [2]:
def active_set_algorithm(Q, c, A, b, x0, tol=1e-6, max_iter=100):

    x = x0
    m, n = A.shape
    active_set = []

    for i in range(max_iter):
        # Check for violated constraints
        residuals = A @ x - b
        violated = np.where(residuals > tol)[0]
        # Compute gradient
        grad = Q @ x + c

        if len(violated) == 0:
            # If there are no violated constraints, check optimality conditions
            if len(active_set) == 0:
                p = -solve(Q, grad)
            else:
                A_eq = A[active_set]
                try:
                    W = np.block([[Q, A_eq.T], [A_eq, np.zeros((len(active_set), len(active_set)))]])
                    rhs = np.block([-grad, np.zeros(len(active_set))])
                    sol = solve(W, rhs)
                    p = sol[:n]
                    lag_mul = sol[n:]
                except np.linalg.LinAlgError:
                    p = np.zeros_like(x)

            # Check for optimality (Lagrange multipliers)
                if np.all(lag_mul >= -tol):
                    print(f'Converged in {i} iterations.')
                    return x

        if np.linalg.norm(p) < tol:
            # Check if Lagrange multipliers are non-negative
            if len(active_set) == 0 or np.all(lag_mul >= -tol):
                print(f'Converged in {i} iterations.')
                return x
            min_lag_mul_idx = np.argmin(lag_mul)
            active_set.pop(min_lag_mul_idx)
        else:
            # Take a step along p
            alpha = 1.0
            active_set_candidate = None
            for j in range(m):
                if j not in active_set:
                    Aj_p = A[j] @ p
                    if Aj_p < 0:
                        # Calculate the step size alpha that would violate this constraint
                        alpha_j = -(A[j] @ x - b[j]) / Aj_p
                        if alpha_j < alpha:
                            alpha = alpha_j
                            active_set_candidate = j
            # Update solution
            x = x + alpha * p

            if active_set_candidate is not None:
                active_set.append(active_set_candidate)
                
    raise ValueError('Active set algorithm did not converge.')

In [3]:
Q = np.array([[2, 0], [0, 2]])
c = np.array([-2, -5])
A = np.array([[-1, 0], [0, -1], [1, 2]])
b = np.array([0, 0, 3])

In [4]:
x0 = np.array([1.5, 0.5])

In [5]:
optimal_x = active_set_algorithm(Q, c, A, b, x0)

Converged in 3 iterations.


In [6]:
print("Optimal solution:", optimal_x)

Optimal solution: [3. 0.]


In [32]:
def maxcut_active_set_algorithm(L, A, b, x0, tol=1e-6, max_iter=100):

    x = x0
    m, n = A.shape
    active_set = []
    lag_mul = None

    for i in range(max_iter):
        residuals = A @ x - b
        violated = np.where(residuals > tol)[0]
        grad = L @ x

        if len(violated) > 0:
            active_set.append(violated[0])

        if len(violated) == 0:
            if len(active_set) == 0:
                # p = -solve(L, grad)
                p, _, _, _ = np.linalg.lstsq(L, -grad, rcond=None)
            else:
                A_eq = A[active_set]
                try:
                    W = np.block([[L, A_eq.T], [A_eq, np.zeros((len(active_set), len(active_set)))]])
                    rhs = np.block([-grad, np.zeros(len(active_set))])
                    # sol = solve(W, rhs)
                    sol, _, _, _ = np.linalg.lstsq(W, rhs, rcond=None)
                    p = sol[:n]
                    lag_mul = sol[n:]
                except np.linalg.LinAlgError:
                    p = np.zeros_like(x)

                if lag_mul is not None and np.all(lag_mul >= -tol):
                    print(f'Converged in {i} iterations.')
                    return x
        else:
            p = np.zeros_like(x)

        if np.linalg.norm(p) < tol:
            if lag_mul is None or np.all(lag_mul >= -tol):
                print(f'Converged in {i} iterations.')
                return x

            min_lag_mul_idx = np.argmin(lag_mul)
            active_set.pop(min_lag_mul_idx)

        else:
            alpha = 1.0
            active_set_candidate = None
            for j in range(m):
                if j not in active_set:
                    Aj_p = A[j] @ p
                    if Aj_p < 0:
                        alpha_j = -(A[j] @ x - b[j]) / Aj_p
                        if alpha_j < alpha:
                            alpha = alpha_j
                            active_set_candidate = j

            x = x + alpha * p

            if active_set_candidate is not None:
                active_set.append(active_set_candidate)
    
    raise ValueError('Active set algorithm did not converge.')

In [49]:
import networkx as nx

G = nx.Graph()
G.add_edge(1, 2)
G.add_edge(1, 3)

In [50]:
L = nx.laplacian_matrix(G)

In [7]:
L = np.array([
    [ 1, -1,  0],
    [-1,  2, -1],
    [ 0, -1,  1]
])

In [52]:
A = np.array([
    [1, 0, 0],
    [-1, 0, 0],
    [0, 1, 0],
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, -1]
])

b = np.array([1, 1, 1, 1, 1, 1])

In [53]:
x0 = np.array([1, 1, 1])

In [28]:
residuals = A @ x0 - b
violated = np.where(residuals > 1e-6)[0]

In [29]:
print(residuals)

[ 0.  -2.  -0.5 -1.5  0.  -2. ]


In [30]:
print(violated)

[]


In [54]:
optimal_x = maxcut_active_set_algorithm(L, A, b, x0)

LinAlgError: 0-dimensional array given. Array must be two-dimensional

In [43]:
print("Optimal solution:", optimal_x)

Optimal solution: [1 1 1]
